In [74]:
import numpy as np
import pandas as pd
import toolsets.data_prep as data_prep
from sklearn.ensemble import RandomForestClassifier
import cleanlab
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_validate
from cleanlab.filter import find_label_issues
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
descriptors = pd.read_csv("data/multiRT/MultiRT_hilic_descriptors.csv", low_memory=False)
descriptors = descriptors.select_dtypes(exclude=['object'])

In [77]:
combined_data = pd.read_csv("data/multiRT/Combined dataset.csv")
hilic = combined_data.loc[combined_data['Column']=='HILIC']
hilic.reset_index(inplace=True, drop=True)

In [78]:
data = pd.concat([hilic, descriptors],axis =1)

In [79]:
data = data_prep.dataset_prep(data)

In [82]:
x, y = data_prep.make_x_y(data)

In [83]:
SEED = 123456
np.random.seed(SEED)
random.seed(SEED)

In [57]:
clf = RandomForestClassifier()

In [84]:
num_crossval_folds = 5
pred_probs = cross_val_predict(
    clf,
    x,
    y,
    cv= num_crossval_folds,
    method="predict_proba",
)

D:\Users\halle\envs\pytorch\lib\site-packages\sklearn\model_selection\_validation.py:1092: RuntimeWarning: Number of classes in training fold (404) does not match total number of classes (460). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(
D:\Users\halle\envs\pytorch\lib\site-packages\sklearn\model_selection\_validation.py:1092: RuntimeWarning: Number of classes in training fold (398) does not match total number of classes (460). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(
D:\Users\halle\envs\pytorch\lib\site-packages\sklearn\model_selection\_validation.py:1092: RuntimeWarning: Number of classes in training fold (350) does not match total number of classes (460). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in prope

In [85]:
ranked_label_issues = find_label_issues(
    labels=y, pred_probs=pred_probs, return_indices_ranked_by="self_confidence"
)

D:\Users\halle\envs\pytorch\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\Users\halle\envs\pytorch\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: Classification metrics can't handle a mix of binary and continuous targets